In [ ]:
!pip install -q pytube
!pip install -q decord
!pip install -q torchviz
!pip install -q transformers
!pip install -q datasets
!pip3 install -q imageio==2.4.1

     |████████████████████████████████| 56 kB 1.7 MB/s 
     |████████████████████████████████| 13.6 MB 209 kB/s 
     |████████████████████████████████| 5.5 MB 4.7 MB/s 
     |████████████████████████████████| 182 kB 59.5 MB/s 
     |████████████████████████████████| 7.6 MB 58.3 MB/s 
     |████████████████████████████████| 451 kB 5.5 MB/s 
     |████████████████████████████████| 115 kB 72.1 MB/s 
     |████████████████████████████████| 212 kB 45.0 MB/s 
     |████████████████████████████████| 127 kB 57.6 MB/s 
     |████████████████████████████████| 3.3 MB 5.1 MB/s 


In [ ]:
# Download from tube
from pytube import YouTube 

# Make a secuence of frames
from decord import VideoReader, cpu
import numpy as np
import math

# View images frames
from PIL import Image

# Images model
import torch
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer

# Sound model
from IPython.display import Audio
import moviepy.editor as mp
import torch
import librosa
import os
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch

# mount drive and gooooo
from google.colab import drive
import pandas as pd

# state bar
from tqdm.auto import tqdm

In [ ]:
# Video model
model_image = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_image.to(device)

max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

# Sound model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model_sound = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")     

Downloading:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/982M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/228 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/241 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Functions

def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)
    start_idx = math.floor(seg_len/clip_len)
    end_idx = start_idx*clip_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

def predict_step(images, model):
  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds

def join_strlist_video(lista):
  entra = "in the video"
  x = []
  for i in range(0,len(lista)):
    if i !=len(lista)-1:
      entra1 = "in the frame {} is {},".format(i+1,lista[i][0])
    else:
      entra1 = "in the frame {} is {}.".format(i+1,lista[i][0])
    x.append(entra1)
  salida = " ".join(x)
  salida = "{} {}".format(entra,salida)
  return salida

# **Loop from here**

In [ ]:
drive.mount('/content/drive')
data_comments = pd.read_csv('/content/drive/My Drive/Hugging Face/datosmulti5.csv')
urls = data_comments["VideoID"].value_counts().to_frame()
urls

Mounted at /content/drive


,VideoID
https://youtu.be/GJ0mO8P37Eg,2819
https://youtu.be/l1heD4T8Yco,874
https://youtu.be/xN9CmeMtSA8,503
https://youtu.be/jAZZTE9t7g0,370
https://youtu.be/OPNZsKgzHGw,322
https://youtu.be/vp0d18LT3os,179
https://youtu.be/RRwqftGrxf4,157
https://youtu.be/8KSzkAR5y7U,135
https://youtu.be/Ac4Bga-WtHQ,118
https://youtu.be/tEg7rAGNfhY,118


In [ ]:
progress_bar = tqdm(range(urls.shape[0]))
descripciones_new = pd.DataFrame()

for url in urls.index.tolist():
  try:
    # youtube to mp4
    yt = YouTube(url)
    streams = yt.streams.filter(file_extension='mp4')
    file_path = streams[0].download()
    videoreader = VideoReader(file_path, num_threads=1, ctx=cpu(0))
    # sample 16 frames
    videoreader.seek(0)
    indices = sample_frame_indices(clip_len=16, frame_sample_rate=6, seg_len=len(videoreader))
    video = videoreader.get_batch(indices).asnumpy()

    # mp4 to mp3
    clip = mp.VideoFileClip(file_path)
    clip.audio.write_audiofile("audio.mp3")

    batch = []
    speech_array, sampling_rate = librosa.load("audio.mp3", sr=16_000)
    sampling_rate = sampling_rate

    input_values = processor(speech_array, sampling_rate=16_000, return_tensors="pt", padding=True)

    ###############

    # predit video
    predict = list()
    for i in range(video.shape[0]):
      predict.append(predict_step(video[i],model_image)) 

    # predit sound
    with torch.no_grad():
        logits = model_sound(input_values.input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    predicted_sentences = processor.batch_decode(predicted_ids)

    ##############

    predict_video = join_strlist_video(predict)
    predict_audio = ' '.join(predicted_sentences)

    description = ''.join([predict_video] + [" The sound says: "] + [predict_audio])

    ##############

    drive.mount('/content/drive')
    descripciones_old = pd.read_excel('/content/drive/My Drive/Hugging Face/descripciones.xlsx')
    descripciones_new["url"] = [url]
    descripciones_new["description"] = [description]
    descripciones_old2 = pd.concat(
        [descripciones_old.reset_index(drop=True),descripciones_new.reset_index(drop=True)]).reset_index(drop=True)

    drive.mount('/drive', force_remount=True)
    descripciones_old2.to_excel('/drive/My Drive/Hugging Face/descripciones.xlsx')

    ##############

    os.remove(file_path)
    os.remove("audio.mp3")
    progress_bar.update(1)
  
  except:
    print(f"SKIPING VIDEO {url}")

  0%|          | 0/14 [00:00<?, ?it/s]

[MoviePy] Writing audio in audio.mp3



100%|██████████| 344/344 [00:01<00:00, 337.34it/s]

[MoviePy] Done.



  warnings.warn("PySoundFile failed. Trying audioread instead.")



...

SKIPING VIDEO https://youtu.be/7tx8oi7sFoc
[MoviePy] Writing audio in audio.mp3



100%|██████████| 228/228 [00:00<00:00, 613.37it/s]

[MoviePy] Done.



  warnings.warn("PySoundFile failed. Trying audioread instead.")



SKIPING VIDEO https://youtu.be/-lcV9WYD6T8
